进行同义词替换：
1. 使用nltk语义库替换valid.json
2. 使用gpt替换valid.json
3. 使用detect model替换test.json，再比较与原test.json文本的差异，使用均方差来评估clean和dirty最终使用auc评分

## 1. 使用nltk

In [ ]:
import json
import nltk
from nltk.corpus import wordnet

# 下载所需的NLTK数据
nltk.download('wordnet')
nltk.download('omw-1.4')

# 同义词替换函数
def synonym_replacement(text):
    words = nltk.word_tokenize(text)
    new_words = []
    for word in words:
        synonyms = wordnet.synsets(word)
        if synonyms:
            synonym = synonyms[0].lemmas()[0].name()
            new_words.append(synonym)
        else:
            new_words.append(word)
    return ' '.join(new_words)

# 加载原始的验证集
input_file_path = 'dataset/valid.json'
output_file_path = 'dataset/synonym_replacement_valid.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 对每个文本进行同义词替换
for entry in data:
    entry['synonym_replacement'] = synonym_replacement(entry['text'])

# 保存到新的json文件
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# 提示操作完成
print("同义词替换已完成并保存到新的json文件中。")


## 2.使用gpt

In [ ]:
import json
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

# 加载GPT-2模型和tokenizer
model_name = 'gpt2'
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

# 设置 pad_token 为 eos_token
tokenizer.pad_token = tokenizer.eos_token

model = GPT2LMHeadModel.from_pretrained(model_name)
model.eval()

# 同义词替换函数
def synonym_replacement_gpt(text, num_return_sequences=1):
    # 编码文本并添加 attention_mask
    inputs = tokenizer.encode_plus(text, return_tensors='pt', padding=True, truncation=True, max_length=1024)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']
    
    # 打印调试信息
    # print(f"Input text: {text}")
    # print(f"Input IDs: {input_ids}")
    # print(f"Input length: {len(input_ids[0])}")
    
    input_length = len(input_ids[0])
    max_new_tokens = 50

    if input_length >= 1024:
        # 如果输入长度已经超过或达到最大长度，则截断输入
        input_ids = input_ids[:, :1024 - max_new_tokens]
        attention_mask = attention_mask[:, :1024 - max_new_tokens]
        input_length = len(input_ids[0])
    
    # 确保生成的总长度不会超过 1024
    max_new_tokens = min(max_new_tokens, 1024 - input_length)
    
    with torch.no_grad():
        outputs = model.generate(
            input_ids,
            attention_mask=attention_mask,
            max_new_tokens=max_new_tokens,
            num_return_sequences=num_return_sequences,
            num_beams=5,
            no_repeat_ngram_size=2,
            early_stopping=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    generated_texts = [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts[0]

# 加载验证集数据
input_file_path = 'dataset/valid.json'
output_file_path = 'dataset/test_synonym_replacement_valid_gpt.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 对每个文本进行同义词替换
# for entry in data:
    # entry['text'] = synonym_replacement_gpt(entry['text'])

# 只对第一个文本进行同义词替换
if data:
    data[0]['text'] = synonym_replacement_gpt(data[0]['text'])

# 保存到新的json文件
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# 提示操作完成
print(f"同义词替换已完成并保存到新的json文件中：{output_file_path}")


## 3.

In [ ]:
import json
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, GPT2Tokenizer, GPT2LMHeadModel
from sklearn.metrics import roc_auc_score
from torch.nn.functional import softmax

# 加载预训练的BERT模型
bert_tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
bert_model = AutoModelForSequenceClassification.from_pretrained('bert-base-uncased')
bert_model.eval()  # 评估模式

# 加载预训练的GPT-2模型用于同义词替换
gpt_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
gpt_model = GPT2LMHeadModel.from_pretrained('gpt2')
gpt_model.eval()

# 同义词替换函数
def synonym_replacement_gpt(text, num_return_sequences=1):
    input_ids = gpt_tokenizer.encode(text, return_tensors='pt')
    with torch.no_grad():
        outputs = gpt_model.generate(
            input_ids, 
            max_length=len(input_ids[0]) + 50, 
            num_return_sequences=num_return_sequences, 
            num_beams=5, 
            no_repeat_ngram_size=2, 
            early_stopping=True
        )
    generated_texts = [gpt_tokenizer.decode(output, skip_special_tokens=True) for output in outputs]
    return generated_texts[0]

# 获取模型输出
def get_model_output(text):
    inputs = bert_tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        outputs = bert_model(**inputs)
    return softmax(outputs.logits, dim=1)

# 对比原始文本和同义改写文本
def compare_texts(original_text, rewritten_text):
    original_output = get_model_output(original_text)
    rewritten_output = get_model_output(rewritten_text)
    return torch.nn.functional.mse_loss(original_output, rewritten_output).item()

# 加载数据
input_file_path = 'dataset/valid.json'  # 替换为你的数据文件路径
output_file_path = 'dataset/synonym_detection_results.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 处理数据
for entry in data:
    entry['synonym_replacement'] = synonym_replacement_gpt(entry['text'])
    entry['difference'] = 、compare_texts(entry['text'], entry['synonym_replacement'])

# 假设阈值可以通过统计分析或其他方法确定
threshold = 0.1
for entry in data:
    entry['predicted_label'] = 1 if entry['difference'] > threshold else 0

# 计算AUC作为评价指标
true_labels = [1 if entry['label'] == 'dirty' else 0 for entry in data]
predicted_labels = [entry['predicted_label'] for entry in data]
auc_score = roc_auc_score(true_labels, predicted_labels)
print(f"AUC Score: {auc_score}")

# 保存结果
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


## 4.textattack

In [1]:
import json
from textattack.augmentation import WordNetAugmenter

# 加载验证集数据
input_file_path = 'dataset/valid.json'
output_file_path = 'dataset/textattack_synonym_replacement_valid.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 初始化WordNetAugmenter
augmenter = WordNetAugmenter()

# 只对第一个文本进行同义词替换
if data:
    original_text = data[0]['text']
    print(f"Original text: {original_text}")
    data[0]['text'] = augmenter.augment(original_text)
    print(f"Replaced text: {data[0]['text']}")

# 保存到新的json文件
with open(output_file_path, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)

# 提示操作完成
print(f"同义词替换已完成并保存到新的json文件中：{output_file_path}")


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\hw4BHJM01\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


Original text: A water company has blamed more people working from home post-pandemic for a new hosepipe ban.South East Water, which supplies more than 2m homes and businesses, will impose the first hosepipe ban of the summer on Monday, affecting households across Kent and Sussex.The company’s chief executive, David Hinton, said that people working from home was a “key factor” behind the ban, as it has “increased drinking water demand”.In a letter to customers, he wrote: “Over the past three years the way in which drinking water is being used across the south-east has changed considerably.“The rise of working from home has increased drinking water demand in commuter towns by around 20% over a very short period, testing our existing infrastructure.”Hinton also blamed low rainfall since April as well as a recent spell of hot weather which he said led to a spike in demand for drinking water.“Our reservoir and aquifer stocks of raw water, essential to our water supply but not ready to be u

## 5.textbolb

In [1]:
import json
from textblob import Word
from textblob import TextBlob

def synonym_replacement_textblob(text):
    blob = TextBlob(text)
    new_words = []
    for word in blob.words:
        synonyms = Word(word).synsets
        if synonyms:
            new_word = synonyms[0].lemmas()[0].name()
            new_words.append(new_word)
        else:
            new_words.append(word)
    return ' '.join(new_words)

# 加载验证集数据
input_file_path = 'dataset/valid.json'
output_file_path = 'dataset/synonym_replacement_first_text_textblob.json'

with open(input_file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 只对第一个文本进行同义词替换
if data:
    original_text = data[0]['text']
    print(f"Original text: {original_text}")
    replaced_text = synonym_replacement_textblob(original_text)
    print(f"Replaced text: {replaced_text}")

    # 保存到新的json文件
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump({"text": replaced_text}, file, ensure_ascii=False, indent=4)

    # 提示操作完成
    print(f"同义词替换已完成并保存到新的json文件中：{output_file_path}")


Original text: A water company has blamed more people working from home post-pandemic for a new hosepipe ban.South East Water, which supplies more than 2m homes and businesses, will impose the first hosepipe ban of the summer on Monday, affecting households across Kent and Sussex.The company’s chief executive, David Hinton, said that people working from home was a “key factor” behind the ban, as it has “increased drinking water demand”.In a letter to customers, he wrote: “Over the past three years the way in which drinking water is being used across the south-east has changed considerably.“The rise of working from home has increased drinking water demand in commuter towns by around 20% over a very short period, testing our existing infrastructure.”Hinton also blamed low rainfall since April as well as a recent spell of hot weather which he said led to a spike in demand for drinking water.“Our reservoir and aquifer stocks of raw water, essential to our water supply but not ready to be u